In [23]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

train=pd.read_csv("/opt/ml/input/data/train/train_ratings.csv")

csv_list = !ls *.csv
csv_list

['EASE@500.csv',
 'FFM@500.csv',
 'GRU4RecF@500.csv',
 'ItemKNN@500.csv',
 'NeuMF@500.csv',
 'Pop@500.csv',
 'SLIMElastic@500.csv']

In [2]:
df=pd.DataFrame()

for csv in csv_list:
    file = pd.read_csv(csv)
    df = pd.concat([df,file])

df.shape

(109487086, 2)

In [3]:
df.drop_duplicates(subset=['user','item'],inplace=True)
df.shape

(34713290, 2)

In [4]:
df = pd.concat([df,train[['user','item']]])
df.shape

(39867761, 2)

In [5]:
# Get a list of all unique users and items
users = df['user'].unique()
items = df['item'].unique()

In [54]:
user2idx = {k:v for v,k in enumerate(sorted(set(train.user)))}
uidx2id = {v:k for v,k in enumerate(sorted(set(train.user)))}
item2idx = {k:v for v,k in enumerate(sorted(set(train.item)))}
itemidx2id = {v:k for v,k in enumerate(sorted(set(train.item)))}

In [6]:
len(users), len(items)

(31360, 6807)

In [7]:
num_users = df['user'].nunique()
num_items = df['item'].nunique()

# Create a 2D array of all possible user-item pairs
pairs = np.array(np.meshgrid(np.arange(num_users), np.arange(num_items))).T.reshape(-1, 2)

In [8]:
# Convert the user and item columns to categorical data
df['user'] = pd.Categorical(df['user'])
df['item'] = pd.Categorical(df['item'])

# Get the categorical codes for the user and item columns
user_codes = df['user'].cat.codes
item_codes = df['item'].cat.codes

In [73]:
user_codes[:4], user_codes[-4:]

(0    0
 1    0
 2    0
 3    0
 dtype: int16,
 5154467    31359
 5154468    31359
 5154469    31359
 5154470    31359
 dtype: int16)

In [64]:
df

,user,item
0,11,4370
1,11,4886
2,11,47
3,11,32587
4,11,40815
...,...,...
5154466,138493,44022
5154467,138493,4958
5154468,138493,68319
5154469,138493,40819


In [87]:
df.item.astype(int).equals(item_codes.map(itemidx2id))

True

In [58]:
user_codes.equals(user_codes2)

False

In [9]:
# Convert the codes to a 2D array of shape (num_interactions, 2)
interactions = np.column_stack((user_codes, item_codes))

In [11]:
nrows, ncols = pairs.shape
dtype={'names':['f{}'.format(i) for i in range(ncols)],
       'formats':ncols * [pairs.dtype]}

In [12]:
C = np.setdiff1d(pairs.view(dtype), interactions.astype(np.int64).view(dtype), assume_unique=True)

In [14]:
# This lpairsst interactionsit is optionpairsl if you're okpairsy with "C" interactionseing pairs structured pairsrrpairsy...
negative_samples = C.view(pairs.dtype).reshape(-1, ncols)

In [18]:
len(negative_samples)

173599759

In [21]:
# Group the negative samples by user
negative_samples_by_user = {}
for user, item in tqdm(negative_samples):
    if user not in negative_samples_by_user:
        negative_samples_by_user[user] = []
    negative_samples_by_user[user].append(item)

100%|████████████████████████████████████████████| 173599759/173599759 [05:05<00:00, 567366.38it/s]


In [75]:
negative_samples_by_user

31360

In [24]:
random.seed(13)

In [93]:
train.user = train.user.map(user2idx)
train.item = train.item.map(item2idx)

In [109]:
neg_df = pd.DataFrame(columns=['user','item'])

In [110]:
for user, items in tqdm(negative_samples_by_user.items()):
    num_items = train[train['user'] == user]['item'].nunique()
    if num_items >= len(items):
        item_list = items
    else:
        item_list = random.sample(items, num_items)
    neg_df.loc[len(neg_df),:]=(user, " ".join(map(str,item_list)))

100%|███████████████████████████████████████████████████████| 31360/31360 [03:28<00:00, 150.39it/s]


In [112]:
b_flat = pd.DataFrame([[i, x] 
               for i, y in neg_df['item'].apply(lambda x: x.split()).iteritems() 
                    for x in y], columns=list('IB'))
b_flat = b_flat.set_index('I')

neg_df2 = neg_df[['user']].merge(b_flat, left_index=True, right_index=True)

In [113]:
neg_df2.columns=['user','item']
neg_df2.user = neg_df2.user.astype(int)
neg_df2.item = neg_df2.item.astype(int)

In [123]:
neg_set=neg_df2.copy()

In [124]:
neg_set.user = neg_set.user.map(uidx2id)
neg_set.item = neg_set.item.map(itemidx2id)

In [132]:
neg_set

,user,item
0,11,97860
0,11,73323
0,11,80864
0,11,4863
0,11,2819
...,...,...
31359,138493,108188
31359,138493,66427
31359,138493,4238
31359,138493,75341


In [126]:
train=pd.read_csv("/opt/ml/input/data/train/train_ratings.csv")

In [135]:
neg_set.reset_index(drop=True, inplace=True)

In [137]:
neg_set.to_pickle('Neg_Sample.pkl')

In [138]:
pd.read_pickle("Neg_Sample.pkl")

,user,item
0,11,97860
1,11,73323
2,11,80864
3,11,4863
4,11,2819
...,...,...
5154016,138493,108188
5154017,138493,66427
5154018,138493,4238
5154019,138493,75341


In [128]:
neg_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5154021 entries, 0 to 31359
Data columns (total 2 columns):
 #   Column  Dtype
---  ------  -----
 0   user    int64
 1   item    int64
dtypes: int64(2)
memory usage: 118.0 MB


In [129]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5154471 entries, 0 to 5154470
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   user    int64
 1   item    int64
 2   time    int64
dtypes: int64(3)
memory usage: 118.0 MB


In [130]:
train[['user','item']].merge(neg_set, on=['user','item'])

,user,item


In [139]:
train2 = train.copy()
neg_set2 = neg_set.copy()

In [140]:
train2['label']=1
neg_set2['label']=0

In [143]:
train2=train2[['user','item','label']]

In [145]:
all_df = pd.concat([train2,neg_set2])

In [148]:
all_df.reset_index(drop=True, inplace=True)

In [149]:
all_df.to_pickle("All_Pos_Neg_Interaction.pkl")

In [150]:
pd.read_pickle('All_Pos_Neg_Interaction.pkl')

,user,item,label
0,11,4643,1
1,11,170,1
2,11,531,1
3,11,616,1
4,11,2140,1
...,...,...,...
10308487,138493,108188,0
10308488,138493,66427,0
10308489,138493,4238,0
10308490,138493,75341,0


In [151]:
all_df.drop_duplicates(subset=['user','item']).shape

(10308492, 3)

In [ ]:
scp -P 2240 All_Pos_Neg_Interaction2.pkl root@27.96.131.55:~
scp -P 2226 All_Pos_Neg_Interaction2.pkl root@101.101.216.53:~